In [151]:
import numpy as np
import pandas as pd
import yaml
import seaborn
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Part 1 - Filter Categories

In [134]:
# Example: 
target_cat = ['Restaurants', 'Food'] # to be continued...

df = pd.read_pickle("./UC01_df_uc_open.p")
print df.shape
df = df[df.categories.apply(lambda x: not set(x).isdisjoint(set(target_cat)))]
print df.shape

(729, 16)
(320, 16)


For now, start from here...

## Part 2 - Build Attribute Table

In [135]:
# load data
df = pd.read_pickle("./UC01_df_uc_open.p") #

df.index = df.business_id.values
att = df.attributes

# extract attributes
a = att.apply(lambda x: yaml.load('['+','.join(x)+']'))

In [136]:
# find full-size attribute set
# if subattribute exists, use '_' to connect them.
att_all = set()
for row in a:
    for i in row:
        if not isinstance(i.values()[0], dict):
            att_all.add(i.keys()[0])
        else:
            prefix = i.keys()[0]
            for k in i.values()[0].iterkeys():
                suffix = k
                temp = prefix + '_' + suffix
                att_all.add(temp)
len(att_all)

81

In [137]:
# create full-size attribute table
# index = business_id
# col = att_all
tab = pd.DataFrame(columns=att_all, index=bid)

for ind in tab.index:
    for j in a[ind]:
        if not isinstance(j.values()[0], dict):
            tab.loc[ind, j.keys()[0]] = j.values()[0]
        else:
            prefix = j.keys()[0]
            for k, v in j.values()[0].iteritems():
                suffix = k
                temp = prefix + '_' + suffix
                tab.loc[ind, temp] = v

## Part 3 - Missing Values 
(missing, not False)

In [146]:
print tab.shape[0]
tab.count(axis=0).sort_values(ascending=False)[1:20]
# 729 * 50% = 360 -> 3 attributes -> RestaurantsPriceRange2 / BusinessParking / BikeParking 

729


HairSpecializesIn_kids               553
HairSpecializesIn_africanamerican    507
RestaurantsDelivery                  507
BikeParking                          507
Music_live                           507
Music_no_music                       507
DietaryRestrictions_dairy-free       454
DogsAllowed                          329
DietaryRestrictions_vegan            300
DietaryRestrictions_kosher           292
HairSpecializesIn_curly              274
BestNights_tuesday                   264
HappyHour                            261
Ambience_classy                      257
BYOBCorkage                          255
BusinessParking_valet                253
Ambience_divey                       252
HairSpecializesIn_coloring           250
DietaryRestrictions_soy-free         250
dtype: int64

In [139]:
print tab.shape[1]
tab.count(axis=1).sort_values(ascending=False)[1:20]

81


L2c-qKZWumCmOCR-dqBLrg    58
jeTfL2kCyBtmFGSrSQHqVw    58
D6SCh4BwNb52wZIqXdS4JQ    57
-Jhlh8Scjy669NdtCfKSSg    57
UUsS7SrTQQ2AvvpslfrvFA    57
uvVOuCbo4_5z_J5WJVG4QQ    56
S_9OaE_RBWBIXnUKvxJKsQ    56
ODURDhURWmZr6b--O4nt3w    56
7B-suS0RHxqxZBBfVfiPLA    56
iZleRh4SOtf4GquA5L8WXw    56
dn9lwYUxmhs_mLKPu7L25Q    56
HbK6IfznbVToEaKVC0WcSg    55
RQxLNvAra_MUUPHdhukilA    55
vqEtZ7CtNB9G8WMzjwJA2g    55
_TWp4gqGF7tQUGloTClcNg    55
mklo4H47YWK7f18YXTCEoA    55
F1qPjasn0R6-j8sa6iYNmA    55
JbnNk6do3aHF3Kr1bX4HzQ    55
pMiz7SP82Tknv1NbVqoSfg    54
dtype: int64

{u'bodies': [<matplotlib.collections.PolyCollection at 0x1135cb310>],
 u'cbars': <matplotlib.collections.LineCollection at 0x113328050>,
 u'cmaxes': <matplotlib.collections.LineCollection at 0x1135cb750>,
 u'cmins': <matplotlib.collections.LineCollection at 0x1135e32d0>}

In [169]:
pl.violinplot(tab.count(axis=1).values,vert=False)

## Part 4 - Join & output

In [140]:
# sort column by alphabeta
tab.columns = tab.columns.sort_values()
# shape
print df.shape, tab.shape
# join two table
df_with_attribute = df.join(tab)

(729, 16) (729, 81)


In [170]:
df_with_attribute.to_pickle("./UC02_df_uc_with_att.p")


## Summary

Label:
* stars

Filter:
* city - u & c
* is_open == 1
* categories <- target category list

Transform:
* attributes

<br/><br/>

Useful Attribute:
* From _attributes_
* review_count
* hours - not preprocess
* lat/long, address, postal_code
* (maybe) name - text?

Not Useful:
* state - all identical
* type - all identical
* neighborhood - all nan

<br/><br/>

Idea from last discussion:
* make full use of remaining categories